In [33]:
from IPython.display import display, Markdown
import polars as pl
import mistune

In [5]:
dataset_pl = pl.read_parquet("../codecontests_cot_sft.parquet")

In [ ]:
all_problems = []
all_completions = []
for i in range(len(dataset_pl)):
    row = dataset_pl[i]
    for j in range(len(row["problem"])):
        all_problems.append(row["problem"][j])
    for j in range(len(row["completions"][0])):
        all_completions.append(row["completions"][0][j])

In [ ]:
for problem, completion in zip(all_problems, all_completions):
    display(Markdown(f"### Problem:"))
    display(problem)
    display(completion)

In [221]:
import pprint
import mistune.renderers
import mistune.renderers.markdown
from collections import defaultdict


markdown_renderer = mistune.create_markdown(renderer=None)

heading_freqs = defaultdict(int)

n_with_steps = 0

def recursive_get_all_children(element, children_list):
    if "children" in element:
        for child in element["children"]:
            recursive_get_all_children(child, children_list)
    else:
        children_list.append(element)
    return children_list

for i in range(10):
    # display(Markdown('## Problem'))
    problem_str = dataset_pl[i]["problem"][0]
    try:
        solution_str = dataset_pl[i]["completions"][0][0]
    except:
        print("warn: no solution")
        continue
    problem_md = markdown_renderer(problem_str)
    solution_md = markdown_renderer(solution_str)
    for i, element in enumerate(solution_md):
        if element["type"] == "heading":
            heading_text = element["children"][0]["raw"].lower()
            has_steps = False
            required_phrases = ["reasoning", "steps", "approach"]
            if (
                any(phrase in heading_text for phrase in required_phrases)
                and not "code" in heading_text
            ):
                has_steps = True
            if has_steps:
                n_with_steps += has_steps
                heading_freqs[heading_text] += 1
                steps_list_idx = i + 1
                if solution_md[i + 1]["type"] == "blank_line":
                    steps_list_idx = i + 2
                steps_list_element = solution_md[steps_list_idx]
                for step in steps_list_element["children"]:
                    # 0 is the prefix + :
                    bullet_point_text = step["children"][1]
                    for sub_step in bullet_point_text["children"]:
                        for sub_sub_step in sub_step["children"]:
                            all_text = recursive_get_all_children(sub_sub_step, [])
                            all_text_str = "".join([t["raw"] for t in all_text])
                            print(all_text_str)

                    
    # display(Markdown(problem_str))
    # display(Markdown('## Solution'))
    # display(Markdown(solution_str))

Use a list of sets to represent the friendships. Each set will store the friends of a particular person.
Use a priority queue (min-heap) to keep track of people who have fewer than k friends in the current group. This helps in efficiently determining who can be added to the group.
Initialize the friendship list with empty sets for each person.
Initialize the group as an empty set.
Initialize the priority queue with all people, since initially, no one has any friends.
For each day, update the friendships by adding the new friendship between the two people.
Re-evaluate the group by checking if the new friendships allow more people to be added to the group.
Use the priority queue to manage the people who have fewer than k friends in the group. If a person now has at least k friends in the group, they can be added to the group.
The size of the group at the end of each day is the maximum number of people that can go on the trip that day.
After processing each day, output the size of the gro

In [213]:
sorted_freqs_table = sorted(heading_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_freqs_table

[('reasoning:', 6),
 ('approach', 4),
 ('solution code', 4),
 ('explanation', 4),
 ('python code:', 4),
 ('explanation:', 4),
 ('solution code:', 2)]